In [1]:
# Check if python is 3.9.5
import sys
print(sys.version)
%load_ext autoreload
%autoreload 2

3.9.5 (default, May 18 2021, 12:31:01) 
[Clang 10.0.0 ]


## Code questions

<div class="alert alert-block alert-warning">
    <b>When do I use <span style="font-family: Monospace; background-color: #FFFFE0">abc.ABC</span> as superclass and when <span style="font-family: Monospace; background-color: #FFFFE0"> abc.ABCMeta</span> as metaclass? </b> <br>
    <a href="https://docs.python.org/3/library/abc.html">Here</a> is the <span style="font-family: Monospace; background-color: #FFFFE0">abc</span> package.
</div>

**Answer:** For us it is not really important.

# Data mining

In [3]:
import multidb_wrapper
import database_classes

In [4]:
from shapely.geometry import Point

In [28]:
naip = database_classes.NAIPData(directory="banasna")

lat = 47.645950
lon = 118.136980

pont = Point(float(lon),float(lat))

In [29]:
naip.run(pont)

logged in
banasna/data_barrel/naip_western_europe_azure/tile_indices/tile_index.dat already exists. It will not be overwritten.
banasna/data_barrel/naip_western_europe_azure/tile_indices/tile_index.idx already exists. It will not be overwritten.
banasna/data_barrel/naip_western_europe_azure/tile_indices/tiles.p already exists. It will not be overwritten.


AssertionError: Location has no intersections with NAIP tiles.

In [21]:
naip.root

'/var/folders/h_/fp7vzm1s46x0fbjfhmjqmlsc0000gn/T/data_barrel'

## NAIP

<div class="alert alert-block alert-danger">
    <b>What exactly is this <a href="https://planetarycomputer.microsoft.com/dataset/naip#Blob-Storage-Notebook">Azure blob storage</a>?</b> <br> It seems, as if it does not need authentication, but we would, if we used <a href="https://planetarycomputer.microsoft.com/dataset/naip#Example-Notebook">Planetary Computer STAC API</a> by Microsoft or even Google's <a href="https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ#description">Earth Engine Data Catalog</a>. Where is the caveat? 
</div>

**Answer:** The explanation was too long to reconstruct for me. But the blob storage is really a path to circumvent the access abrrier in this case.

<div class="alert alert-block alert-danger">
    <b>Are the NAIP tiles actually overlapping?</b> <br> I am unsure, whether a location query could lead to multiple tiles within a given year.
</div>

In [6]:
multidb_wrapper.set_directory("aa")

'aa/data_barrel'

Okay, how should I proceed?
- Implement the date in the query! This reduces the images retrieved to maybe one.
- Find the most centered image and start to stitch
- start off with fixed raster size
- use rasterio and fiona
- 

In [20]:
multidb_wrapper.set_locations([1,2,2,3], [2,3,4])

AssertionError: The lists for latitudes ({len(longitudes)}) and lon (4) have unequal length. Please adjust.

---